# Hadoop Streaming assignment 1: Words Rating

The purpose of this task is to create your own WordCount program for Wikipedia dump processing and learn basic concepts of the MapReduce.

In this task you have to find the 7th word by popularity and its quantity in the reverse order (most popular first) in Wikipedia data (`/data/wiki/en_articles_part`).

There are several points for this task:

1) As an output, you have to get the 7th word and its quantity separated by a tab character.

2) You must use the second job to obtain a totally ordered result.

3) Do not forget to redirect all trash and output to /dev/null.

Here you can find the draft of the task main steps. You can use other methods for solution obtaining.

## Step 1. Create mapper and reducer.

<b>Hint:</b>  Demo task contains almost all the necessary pieces to complete this assignment. You may use the demo to implement the first MapReduce Job.

In [1]:
%%writefile mapper1.py

# Your code for mapper here.
import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Writing mapper1.py


In [2]:
%%writefile reducer1.py

# Your code for reducer here.
import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Writing reducer1.py


In [0]:
# You can use this cell for other experiments: for example, for combiner.

## Step 2. Create sort job.

<b>Hint:</b> You may use MapReduce comparator to solve this step. Make sure that the keys are sorted in ascending order.

In [0]:
# Your code for sort job here. Don't forget to use magic writefile

In [6]:
%%writefile mapper2.py

import sys
reload(sys)

for line in sys.stdin:
    try:
        word, count = line.strip().split('\t', 1)
        count = int(count)
        print '%d\t%s' % (count, word)
    except ValueError as e:
        continue

Overwriting mapper2.py


In [7]:
%%writefile reducer2.py

import sys
reload(sys)

for line in sys.stdin:
    try:
        count, word = line.strip().split('\t', 1)
        count = int(count)
        print '%s\t%d' % (word, count)
    except ValueError as e:
        continue

Overwriting reducer2.py


## Step 3. Bash commands

<b> Hint: </b> For printing the exact row you may use basic UNIX commands. For instance, sed/head/tail/... (if you know other commands, you can use them).

To run both jobs, you must use two consecutive yarn-commands. Remember that the input for the second job is the ouput for the first job.

In [9]:
%%bash

OUT_DIR="assignment1_wordcount_"$(date +"%s%6N")

NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

# Code for your first job
# yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar ...
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper1.py,reducer1.py \
    -mapper "python mapper1.py" \
    -combiner "python reducer1.py" \
    -reducer "python reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

OUT_DIR1="assignment1_sorting_"$(date +"%s%6N")

NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR1} > /dev/null

# Code for your second job
# yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar ...
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming sorting" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-nr" \
    -files mapper2.py,reducer2.py \
    -mapper "python mapper2.py" \
    -reducer "python reducer2.py" \
    -input ${OUT_DIR} \
    -output ${OUT_DIR1} > /dev/null

# Code for obtaining the results
hdfs dfs -cat ${OUT_DIR1}/part-00000 | sed -n 7p
hdfs dfs -rm -r -skipTrash ${OUT_DIR1}* > /dev/null

is	126420


rm: `assignment1_wordcount_1534486609377997': No such file or directory
18/08/17 06:16:53 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/08/17 06:16:53 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/08/17 06:16:54 INFO mapred.FileInputFormat: Total input files to process : 1
18/08/17 06:16:54 INFO mapreduce.JobSubmitter: number of splits:2
18/08/17 06:16:54 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1534370228375_0013
18/08/17 06:16:54 INFO impl.YarnClientImpl: Submitted application application_1534370228375_0013
18/08/17 06:16:55 INFO mapreduce.Job: The url to track the job: http://94eb3f8bc528:8088/proxy/application_1534370228375_0013/
18/08/17 06:16:55 INFO mapreduce.Job: Running job: job_1534370228375_0013
18/08/17 06:17:02 INFO mapreduce.Job: Job job_1534370228375_0013 running in uber mode : false
18/08/17 06:17:02 INFO mapreduce.Job:  map 0% reduce 0%
18/08/17 06:17:18 INFO mapreduce.Job:  map 28% reduce 0%
18/08/17